In [4]:
import numpy as np
from fl.preprocessing import load_mnist, data_to_client
from fl.model import NN
from fl.utils import plot_train_and_test, weights_to_json
from fl.federated_learning import federated

# Challenge 2 : Du poison [2/2]

![du_poison_2.jpg](https://i.imgflip.com/8nfix9.jpg)

Cette fois, une défense a été mise en place. Il s'agit d'éviter qu'un client ait trop d'influence en forçant une variation maximale pour les poids. Mais *est-ce suffisant ?* 

## Récupération du drapeau 

Comme pour le premier challenge, il faut envoyer ses poids à l'API. 

In [5]:
epochs = 5
epochs_fl = 1
nb_clients = 5

x_train, y_train, x_test, y_test = load_mnist()
x_clients, y_clients = data_to_client(x_train, y_train, nb_clients=nb_clients)

In [25]:

class EmpoisonneurNN(NN):
    def __init__(self, **kwargs):
        super(EmpoisonneurNN, self).__init__(**kwargs)

    def get_weights(self):
        # On récupère les poids normaux du modèle
        original_weights = super(EmpoisonneurNN, self).get_weights()

        # Empoisonnement ciblé et subtil
        poisoned_weights = []
        for weight in original_weights:
            # Augmenter subtilement les poids en ajoutant une petite valeur
            increment = np.random.uniform(0.04, 0.1, weight.shape)
            poisoned_weight = weight + increment

            poisoned_weights.append(poisoned_weight)

        return poisoned_weights

    def set_weights(self, weights):
        # Appliquer une logique d'empoisonnement identique ici
        poisoned_weights = []
        for weight in weights:
            increment = np.random.uniform(0.01, 0.1, weight.shape)
            poisoned_weight = weight + increment
            poisoned_weights.append(poisoned_weight)

        super(EmpoisonneurNN, self).set_weights(poisoned_weights)

# Utilisation de l'EmpoisonneurNN
model = EmpoisonneurNN()



In [ ]:
federated_learning = federated(
    NN(),
    x_clients, 
    y_clients, 
    x_test, 
    y_test, 
    fl_iterations=epochs,
    epochs_fl=epochs_fl
)

In [8]:
import requests as rq

URL = "https://du-poison.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

{'message': 'Statut : en pleine forme !'}

In [14]:
d = weights_to_json(model.get_weights())
rq.post(URL + "/challenges/1", json=d).json()

{'message': "Raté ! Le score de l'apprentissage fédéré est de 0.942. Il faut l'empoisonner pour qu'il passe en dessous de 0.5"}

In [26]:
d = weights_to_json(model.get_weights())
rq.post(URL + "/challenges/2", json=d).json()

{'message': 'Bravo ! Voici le drapeau : 404CTF{p3rF0rm4nc3_Ou_s3cUR1T3_FaUt_iL_Ch01s1r?}'}